# Transhipment: transportation

## Introduction to optimization and operations research

Michel Bierlaire


In [ ]:

import numpy as np
from matplotlib import pyplot as plt
from networkx import (
    DiGraph,
    get_node_attributes,
    draw_networkx_nodes,
    draw_networkx_labels,
    draw_networkx_edges,
    draw_networkx_edge_labels,
)
from teaching_optimization.simplex_tableau import SimplexAlgorithmTableau
from teaching_optimization.tableau import SimplexTableau


There are about 45000 Tesla Model S produced per year, that need
batteries.  30000 come from the Gigafactory factory in the Nevada, and
15000 from the Gigafactory in Shanghai. Each battery requires 63kg of
lithium. Consequently, the company purchases 2835 tons of lithium per
year: 1927 tons from mines in Australia, 605 tons from Chili and 303
tons from China.

The transportation costs from the mining sites to the factories are
reported in the following table:

|             | Australia   |  Chile      |   China     |
|:-----------:|:-----------:|:-----------:|:-----------:|
| Nevada      |      8      |      6      |       7     |
| Shanghai    |      4      |      12     |       1     |

How should the transportation be organized to supply the factories at
minimum cost?

- Model the problem as a transhipment problem,
- draw the corresponding network, and
- solve the optimization problem using the simplex algorithm.

The problem is not associated with a physical network. Still, we
define a network in the following way. We define a node for each
factory, represented on the right side of the network below, and a
node for each mine, represented on the left side.

We introduce an arc between each pair of mine and factory. For
example, the arc from Australia to Nevada means that the Australian
mine is providing lithium to the Nevada factory.  The cost associated
with each arc is the transportation price per ton.


A transhipment problem is written
$$
\min_{x \in \mathbb{R}^n} \sum_{(i,j)\in \mathcal{A}}c_{ij} x_{ij}.
$$
subject to the flow conservation constraints,
$$
\operatorname{div}(x)_i = s_i \; \forall i \in \mathcal{N},
$$
and the bounds on the flows:
$$
\ell_{ij} \leq x_{ij} \leq u_{ij}, \; \forall (i,j) \in \mathcal{A}.
$$

In our case,

- $\mathcal{N}$ is the set of nodes: Australia (A), Chile (L),
China (C), Nevada (N), Shanghai (S).
- $\mathcal{A}$ is the set of possible transactions, that is any
pair mine, factory.
- Costs: $c_{ij}$, $\forall (i,j) \in \mathcal{A}$ corresponding
to transportation prices per ton, as reported in the problem
description.
- Supply: For each mine, the supply in tons is provided in the
problem description: 1927 for Australia, 605 for Chile and 303 for
China. The demand in lithium for Nevada is $30000 \cdot 0.063 =
1890$ tons. For Shanghai, it is $15000 \cdot 0.063 = 945$ tons of
lithium. For each factory, the supply is the demand with the
opposite sign, that is -1890 for Nevada, -945 for Shanghai.
- Bounds: As the transactions can only occur in one direction, we
impose $x_{ij} \geq 0$, $\forall (i,j) \in \mathcal{A}$. No upper
bound is necessary.

The transhipment problem is
$$
\min_{x\in\mathbb{R}^6} 8 x_{AN} + 4 x_{AS} + 6 x_{LN} + 12 x_{LS} + 7 x_{CN}+
x_{CS},
$$
subject to
\begin{align*}
x_{AN} + x_{AS} &= 1927, \\ x_{LN} + x_{LS} &= 605, \\ x_{CN} +
x_{CS} &= 303, \\ -x_{AN} - x_{LN} - x_{CN} &= -1890, \\ - x_{AS} -
x_{LS} - x_{CS} &= -945, \\ x_{ij} &\geq 0,& \forall (i,j) \in
\mathcal{A}.
\end{align*}

Create the directed graph

In [ ]:
the_network = DiGraph()


Add nodes with data

In [ ]:
supply_australia = 1927
the_network.add_node('Australia', supply=supply_australia)
supply_chile = 605
the_network.add_node('Chile', supply=supply_chile)
supply_china = 303
the_network.add_node('China', supply=supply_china)
supply_nevada = -1890
the_network.add_node('Nevada', supply=supply_nevada)
supply_shanghai = -945
the_network.add_node('Shanghai', supply=supply_shanghai)



Add arcs with data

In [ ]:
the_network.add_edge('Australia', 'Nevada', cost=8)
the_network.add_edge('Australia', 'Shanghai', cost=4)
the_network.add_edge('Chile', 'Nevada', cost=6)
the_network.add_edge('Chile', 'Shanghai', cost=12)
the_network.add_edge('China', 'Nevada', cost=7)
the_network.add_edge('China', 'Shanghai', cost=1)


Define positions for each node

In [ ]:
pos = {
    'Australia': (1, 0),
    'Chile': (1, -5),
    'China': (1, -10),
    'Nevada': (10, -2.5),
    'Shanghai': (10, -7.5),
}


Draw the nodes with labels (including supply data)

In [ ]:
node_labels = {
    node: f"{node}\nSupply: {data['supply']}"
    for node, data in the_network.nodes(data=True)
}



Draw the arcs with labels (including cost data)

In [ ]:
edge_labels = {(u, v): f"Cost: {d['cost']}" for u, v, d in the_network.edges(data=True)}



Display the graph

In [ ]:
draw_networkx_nodes(
    the_network,
    pos,
    node_size=3000,
    node_color='lightblue',
    edgecolors='black',
    alpha=0.5,
)
draw_networkx_labels(the_network, pos, labels=node_labels, font_size=8)
draw_networkx_edges(the_network, pos, edgelist=the_network.edges(), arrows=True)
draw_networkx_edge_labels(
    the_network, pos, edge_labels=edge_labels, font_size=8, label_pos=0.3
)
plt.show()


Solve now the problem with the simplex algorithm.

In order to solve the problem with the simplex algorithm let's first modify it such that the right-hand
side contains
only non negative values by multiplying the corresponding constraints by $-1$ (it is not strictly
necessary, as the
algorithm can take care of it automatically).  If we do
so, and number the variables in the following way:

$$\begin{align*}
x_1 &= x_{AN}, & x_2 &= x_{AS}, \\ x_3 &= x_{LN}, & x_4 &= x_{LS},
\\ x_5 &= x_{CN}, & x_6 &= x_{CS},
\end{align*}$$
we obtain the following data:

$$ A= \begin{pmatrix*} 1 & 1 & 0 & 0 & 0 & 0 \\ 0 & 0 & 1 & 1 & 0 & 0
\\ 0 & 0 & 0 & 0 & 1 & 1 \\ 1 & 0 & 1 & 0 & 1 & 0 \\ 0 & 1 & 0 & 1 &
0 & 1,
\end{pmatrix*}$$
$$b= \left(\begin{array}{c} 1927 \\ 605 \\ 303 \\ 1890 \\ 945 \end{array}\right) $$

and

$$c^T = \begin{pmatrix} 8 & 4 & 6 & 12 & 7 & 1 \end{pmatrix}.$$

In [ ]:
matrix_a = np.array(
    [
        [1, 1, 0, 0, 0, 0],
        [0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 1, 1],
        [1, 0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0, 1],
    ]
)


In [ ]:
vector_b = np.array([1927, 605, 303, 1890, 945])


In [ ]:
vector_c = np.array([8, 4, 6, 12, 7, 1])


We create the algorithm

In [ ]:
the_algorithm = SimplexAlgorithmTableau(
    objective=vector_c,
    constraint_matrix=matrix_a,
    right_hand_side=vector_b,
)


We solve the problem

In [ ]:
optimal_tableau: SimplexTableau = the_algorithm.solve()


Check if the problem is feasible

In [ ]:
if optimal_tableau is None:
    print(f'Optimization problem is infeasible.')
    sys.exit()


Optimal solution

In [ ]:
print(optimal_tableau.feasible_basic_solution)


Interpretation in terms of arc flows:

In [ ]:
flow_a_n = optimal_tableau.feasible_basic_solution[0]
print(f'Australia -> Nevada: {flow_a_n}')

flow_a_s = optimal_tableau.feasible_basic_solution[1]
print(f'Australia -> Shanghai: {flow_a_s}')

flow_l_n = optimal_tableau.feasible_basic_solution[2]
print(f'Chile -> Nevada: {flow_l_n}')

flow_l_s = optimal_tableau.feasible_basic_solution[3]
print(f'Chile -> Shanghai: {flow_l_s}')

flow_c_n = optimal_tableau.feasible_basic_solution[4]
print(f'China -> Nevada: {flow_c_n}')

flow_c_s = optimal_tableau.feasible_basic_solution[5]
print(f'China -> Shanghai: {flow_c_s}')


Optimal value

In [ ]:
print(f'{optimal_tableau.value_objective_function}')